In [ ]:
### Preparing the Wrold Cup Workload
import sklearn as sl
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


csv_file = 'https://raw.githubusercontent.com/nimamahmoudi/faas-keep-alive-modeling/master/datasets/wc98/invocation_count.csv'
df = pd.read_csv(csv_file, index_col=0, parse_dates=True)
f = plt.figure(figsize=(10,12))
df['1998-06-30 08:00:01':'1998-07-01 08:00:00'].plot()
users = df['1998-06-30 08:00:01':'1998-07-01 08:00:00'].values
plt.subplot(211)
plt.plot(users)
scaling = sl.preprocessing.MinMaxScaler()
scaled_users = np.ceil(120*scaling.fit_transform(users)).astype(int)
plt.subplot(212)
plt.plot(scaled_users)
plt.show(block = True)


In [ ]:
#### Importing Necessary Libraries
import tensorflow as tf
import tensorflow.contrib.layers as layers
import os
import subprocess
import time
import sys
import time
import math
from kubernetes import client, config
tf.reset_default_graph()  # This is to reset the graph
output = subprocess.check_output('kubectl get pods', shell=True)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/jupyter/google.json"
dir_path = os.path.dirname(os.path.realpath('__file__'))


##############
## PID Parameters
KP = 0.0004
KI = 0.0004
KD = 0.00005
r = 800
e = np.zeros([3,1])
resp=np.zeros([5,1])
rps=np.zeros([5,1])
cont_rq=np.zeros([5,1])
cont_rd=np.zeros([5,1])
users=np.zeros([5,1])
e_pre = np.zeros([3,1])
x_PID_old = np.zeros([3,1])
x_PID_new = np.zeros([3,1])
eta_controller = 2e-12
eta_prev = eta_controller
max_input_user = 1
min_input_user = 0
max_input_cont = 1
min_input_cont = 0
max_output_resp = 1
min_output_resp = 0
max_rps = 1
min_rps = 0
min_users = 0
max_users = 1
resp[0] = 0
resp[1] = 0
resp[2] = 0
resp[3] = 0
resp[4] = 0
cont_rq[0]=1
cont_rq[1]=1
cont_rq[2]=1
cont_rq[3]=1
cont_rq[4]=1
cont_rd[0]=1
cont_rd[1]=1
cont_rd[2]=1
cont_rd[3]=1
rps[0] = 0
rps[1] = 0
rps[2] = 0
rps[3] = 0
rps[4] = 0
fil_resp = np.zeros([5,1]) 
############## 
# Neural Network Parameters
n_hidden = 40
n_classes = 1
n_input = 9
eta = 1e-4
y_train=np.zeros([1,1])
x_train=np.zeros([1,1])

############## 
#### Defining the Neural Network Model Identifier, one can add more layers to make it deeper 

def multilayer_perceptron(x):
    fc1=tf.layers.dense(x, n_hidden,activation=tf.nn.tanh,name="FL")
    #fcd=tf.layers.dropout(inputs=fc1,rate=0.5,training=True)
    #fc2=tf.layers.dense(fc1, 10,activation=tf.nn.elu,name="FL2")
    #fcd2=tf.layers.dropout(inputs=fc2,rate=0.1,training=True)
    #fcd2=tf.layers.dropout(inputs=fc2,rate=0.5)
    #fc3=tf.layers.dense(fc2, 10,activation=tf.nn.elu,name="FL3")
    #fcd3=tf.layers.dropout(inputs=fc3,rate=0.1,training=True)
    #fc3=layers.fully_connected(fc2, 20,activation_fn=tf.nn.elu,scope='FC3')
    out=tf.layers.dense(fc1,n_classes,activation=None,name="OL") 
    return out

# build model, loss, and train op
x = tf.placeholder(tf.float32, [None, n_input], name='placeholder_x')
y = tf.placeholder(tf.float32, [None, n_classes], name='placeholder_y')
y_hat = multilayer_perceptron(x)
loss = tf.reduce_mean(tf.square(y-y_hat))
train = tf.train.AdamOptimizer(learning_rate= eta).minimize(loss)
init = tf.global_variables_initializer()

sess = tf.Session()
#sess.run(init)
saver = tf.train.Saver()
saver.restore(sess, dir_path + "/Model-small-new.ckpt")
config.load_kube_config()
#%%
DEPLOYMENT_NAME = 'wordpress'
DEPLOYMENT_NS = 'default'


api_instance = client.ExtensionsV1beta1Api()

def get_replica_and_ready():
    api_response = api_instance.read_namespaced_deployment(DEPLOYMENT_NAME, DEPLOYMENT_NS)
    return api_response.status.replicas, api_response.status.ready_replicas

def set_replica_num(rnum):
    rnum = int(rnum)
    if rnum < 1:
        rnum = 1
    api_response = api_instance.read_namespaced_deployment(DEPLOYMENT_NAME, DEPLOYMENT_NS)
    api_response.spec.replicas = rnum
    api_instance.patch_namespaced_deployment_scale(DEPLOYMENT_NAME, DEPLOYMENT_NS, api_response)
u_pre = 6
set_replica_num(u_pre)
print(get_replica_and_ready())


In [ ]:
#%% Change working directory from the workspace root to the ipynb file location. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
try:
    os.chdir(os.path.join(os.getcwd(), 'examples'))
    print(os.getcwd())
except:
    pass
import pacs_load_tester as load_tester
moshtagh = np.zeros((1,13))

#%%
def custom_sensing():
    r1,r2 = get_replica_and_ready()
    return {'r1':r1 , 'r2':r2 , 'KP':KP, 'KI':KI , 'KD':KD}
lt = load_tester.pacsLoadTester(hatch_rate=1, temp_stat_max_len=5, base='http://35.232.158.172:8089/')

lt.custom_sensing = custom_sensing
lt.custom_sensing()
from tqdm.auto import tqdm
tqdm.pandas()

loop_timer = load_tester.TimerClass()
total_timer = load_tester.TimerClass()
user_sequence = [30,60,30,60,30]

lt.custom_sensing = custom_sensing
lt.change_count(user_sequence[0])
lt.start_capturing()

loop_time_in_secs = load_tester.get_loop_time_in_secs('20s')

loop_timer.tic()
total_timer.tic()
results = None
#old_response = 300
for i in tqdm(range(len(user_sequence)*36)):
    print("U(t)  : ", u_pre)
    user_count = user_sequence[math.floor(i/36)]
    lt.change_count(user_count)
    sleep_time = loop_time_in_secs - loop_timer.toc()
    if sleep_time > 0:
        time.sleep(sleep_time)

    loop_timer.tic()

    result = lt.get_all_stats()
    df_result = pd.DataFrame(data=result)

    if results is None:
        results = df_result
    else:
        results = results.append(df_result)


    #####Identification
    if (result['current_response_time_average'])!=[]:
        if result['current_response_time_average'][-1]!=None :
            cont_rq[0] =  np.nanmean(np.array(np.ceil(u_pre)))
            
            #cont_rq[0] = u_pre
            cont_rq[1] = (cont_rq[1]-min_input_cont)/(max_input_cont-min_input_cont)
            cont_rq[2] = (cont_rq[2]-min_input_cont)/(max_input_cont-min_input_cont)
            cont_rq[3] = (cont_rq[3]-min_input_cont)/(max_input_cont-min_input_cont)
            cont_rq[4] = (cont_rq[4]-min_input_cont)/(max_input_cont-min_input_cont)
            cont_rd[0]= np.nanmean(np.array(result['custom_r2'][-1]))
            resp[0] = (np.nanmean(np.array(result['current_response_time_average'],dtype=float))-min_output_resp)/(max_output_resp-min_output_resp) 
            resp[1] =(resp[1]-min_output_resp)/(max_output_resp-min_output_resp)
            resp[2] =(resp[2]-min_output_resp)/(max_output_resp-min_output_resp)
            resp[3] =(resp[3]-min_output_resp)/(max_output_resp-min_output_resp)
            resp[4] =(resp[4]-min_output_resp)/(max_output_resp-min_output_resp)
            fil_resp[0:4] = resp[0:4]
            rps[0] = (np.nanmean(np.array(result['current_rps'][-1],dtype=float))-min_rps)/(max_rps-min_rps)
            rps[1] = (rps[1]-min_rps)/(max_rps-min_rps)
            rps[2] = (rps[2]-min_rps)/(max_rps-min_rps)
            rps[3] = (rps[3]-min_rps)/(max_rps-min_rps)
            rps[4] = (rps[4]-min_rps)/(max_rps-min_rps)

            users[0] = (np.nanmean(np.array(result['user_count'],dtype=float))-min_users)/(max_users-min_users)
            users[1] = (users[1]-min_users)/(max_users-min_users)
            users[2] = (users[2]-min_users)/(max_users-min_users)
            users[3] = (users[3]-min_users)/(max_users-min_users)
            users[4] = (users[4]-min_users)/(max_users-min_users)
            r_normalized = (r-min_output_resp)/(max_output_resp-min_output_resp)
            batch_x = np.reshape([cont_rq[0]/40,cont_rq[1]/40,cont_rq[2]/40,cont_rd[1]/40,cont_rd[2]/40,cont_rd[3]/40,users[0]/150,users[1]/150,users[2]/150],(1,9))
            batch_y = np.reshape(resp[0],(1,1))
            print("batch_x : ",batch_x)
            for epoch in range(1):
                epoch_loss = 0.0
                batch_steps =1
                for i in range(batch_steps):
                    _, c = sess.run([train, loss],
                                       feed_dict={x: batch_x, y: batch_y})
                    epoch_loss += c / batch_steps
                    print ('Epoch %02d, Loss = %.12f , ' % (i, epoch_loss))
                    y_now = sess.run(y_hat, feed_dict={x: batch_x})
                    print("PREDICTION : ", y_now)

            e_pre[0] = e[0]
            e_pre[1] = e[1]
            e_pre[2] = e[2]
            e[0]= -(r_normalized - resp[0])
            if e[0]>=-300 and e[0]<=0:
                e[0] = 0
            e[1]= e_pre[0]
            e[2]= e_pre[1]
            x_PID_old[0] = e_pre[0] - e_pre[1]
            x_PID_old[1] = e_pre[0]  
            x_PID_old[2] = e_pre[0] - 2*e_pre[1] + e_pre[2]
            x_PID_new[0] = e[0] - e[1]
            x_PID_new[1] = e[0]  
            x_PID_new[2] = e[0] - 2*e[1] + e[2]            
            print("Error (t+1) : ",e[0], "Error (t) : ", e[1],"Error (t-1) : ", e[2])

            dy_du = tf.gradients(y_hat,x)
            sensitivity= sess.run(dy_du,feed_dict={x:batch_x})
            if np.abs(resp[0]-y_now)>=300 :
                eta_controller = 0 
            else :
                eta_controller = eta_prev
            delta_KP = 0.1*eta_controller*e[0]*sensitivity[0][0][0]*x_PID_old[0] 
            delta_KI = 0.1*eta_controller*e[0]*sensitivity[0][0][0]*x_PID_old[1] 
            delta_KD = 0.01*eta_controller*e[0]*sensitivity[0][0][0]*x_PID_old[2] 
            KP = KP - delta_KP
            KI = KI - delta_KI
            KD = KD - delta_KD
            cont_rd[3]=cont_rd[2]
            cont_rd[2]=cont_rd[1]
            cont_rd[1]=cont_rd[0]
            cont_rq[4]= cont_rq[3]
            cont_rq[3]= cont_rq[2]
            cont_rq[2]= cont_rq[1]
            cont_rq[1]= cont_rq[0]
            resp[4]=resp[3]
            resp[3]=resp[2]
            resp[2]=resp[1]
            resp[1]=resp[0]
            rps[4] = rps[3]
            rps[3] = rps[2]
            rps[2] = rps[1]
            rps[1] = rps[0]
            users[4] = users[3]
            users[3] = users[2]
            users[2] = users[1]
            users[1] = users[0]
            if u_pre ==40 or u_pre < 1:
                x_PID_new[1] = 0  
            u_pre = u_pre + KP*x_PID_new[0] + KI*x_PID_new[1] + KD*x_PID_new[2]
            print("u(t+1) : ", u_pre)
            default = u_pre
            
            if np.isnan(u_pre):
                set_replica_num(np.floor(default))
            if u_pre >= 40 :
                u_pre = 40
                try:
                    set_replica_num(40)
                except:
                    print("Error occured !!!!!")
            elif u_pre <= 1 :
                set_replica_num(1)
                u_pre = 1
            else :
                try : 
                    set_replica_num(np.ceil(u_pre))
                except : 
                    print("Exception !!!")
            print("KP : ",KP," KI : ",KI, "KD : ",KD, "Control Input : ",u_pre, "Modeling Error : ",(resp[0]-y_now)," Gradient : ",sensitivity[0][0][0])
            print("X1 old: ",x_PID_old[0], "X2 old: ",x_PID_old[1], "X3 old: ",x_PID_old[2])
            print("X1 new: ",x_PID_new[0], "X2 new: ",x_PID_new[1], "X3 new: ",x_PID_new[2])
            print("delta_KP :",delta_KP,"delta_KI :",delta_KI,"delta_KD :",delta_KD)
        print("iteration : ",i ,"average RSPT is: ",np.nanmean(np.array(result['current_response_time_average'],dtype=float)),"Number of Replica Ready: ",result['custom_r2'][-1], "Number of Users: ",user_count)
        print(" ERROR : ",e[0])
        print("----------------------------------------------")
        
lt.stop_test()

results, filename = lt.prepare_results_from_df(results)

results.head()

res = results
res = res[res['total_rps'] > 0]
responsetime_avg = np.nanmean(np.array(res['current_response_time_average'],dtype=float))
responsetime_var = np.sqrt(np.var(res['current_response_time_average']))
container_avg = np.average(res['custom_r2'])
container_var = np.sqrt(np.var(res['custom_r2']))
print("average res : ", responsetime_avg , "Var Res : ",responsetime_var, "average Container : ",container_avg, "Var Container : ", container_var )
import matplotlib.pyplot as plt

get_ipython().run_line_magic('matplotlib', 'inline')

plt.figure(figsize=(60,30),dpi=200)
plt.subplot(511)
#plt.plot(res['elapsed_min'], res['min_response_time'], label='min_response_time')
#plt.plot(res['elapsed_min'], res['current_response_time_percentile_50'], label='median_response_time')
plt.plot(res['elapsed_min'], res['current_response_time_average'], label='avg_response_time')
plt.plot(res['elapsed_min'],r*np.ones(len(res['current_response_time_average'])), label='setpoint')
#plt.plot(res['elapsed_min'], res['current_response_time_percentile_95'], label='95th percentile')
#plt.plot(res['elapsed_min'], res['max_response_time'], label='max_response_time')
plt.ylim((0,6000))
plt.xlabel('Time (minutes)')
plt.ylabel('Average Response Time (ms)')
plt.legend()

plt.subplot(512)
plt.plot(res['elapsed_min'], res['user_count'])
plt.xlabel('Time (minutes)')
plt.ylabel('Num of Users')

plt.subplot(513)
plt.plot(res['elapsed_min'], res['total_rps'])
plt.xlabel('Time (minutes)')
plt.ylabel('Throughput (req/s)')

plt.subplot(514)
plt.plot(res['elapsed_min'], res['fail_ratio'])
plt.xlabel('Time (minutes)')
plt.ylabel('Fail Ratio')

plt.subplot(515)
plt.plot(res['elapsed_min'], res['custom_r1'])
plt.plot(res['elapsed_min'], res['custom_r2'])
plt.xlabel('Time (minutes)')
plt.ylabel('Custom Value')



filename = filename.replace('aa.csv', '')
plt.savefig(filename + '.png', dpi=300)
plt.savefig(filename + '.pdf')
plt.show()


